In [1]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [3]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "9u20HxRldvsvMO8efF7XoajMG"
    consumer_secret = "OMXcTO72b7K1lNTttvxCXNPKrtAS3LWF3gnfbSlsTAPwA3yD0n"
    access_token = "1250831708380790784-cmlpbgOr11s5hLpH0hYSpR7mhXDwyh"
    access_token_secret = "vIoHLClsaB4ybVApK4ANmzvcXFXKrpKqyXlOpYHrtwNWv"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)
    stream.filter(track=['coronavirus'], is_async=True)

{'created_at': 'Thu Apr 23 21:17:53 +0000 2020', 'id': 1253432932372410369, 'id_str': '1253432932372410369', 'text': 'I been playing this game for 20 years lol', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1048269998744133632, 'id_str': '1048269998744133632', 'name': 'Jacob Randall', 'screen_name': 'JacobRanda11', 'location': 'South Carolina, USA', 'url': None, 'description': 'Loved by The King.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 98, 'friends_count': 143, 'listed_count': 0, 'favourites_count': 1604, 'statuses_count': 210, 'created_at': 'Fri Oct 05 17:53:38 +0000 2018', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': Fa

{'created_at': 'Thu Apr 23 21:17:53 +0000 2020', 'id': 1253432933521612807, 'id_str': '1253432933521612807', 'text': '@Bombitznyc I’m finna open up my own shìt at this point', 'display_text_range': [12, 55], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1253428909099515904, 'in_reply_to_status_id_str': '1253428909099515904', 'in_reply_to_user_id': 723601584, 'in_reply_to_user_id_str': '723601584', 'in_reply_to_screen_name': 'Bombitznyc', 'user': {'id': 2464393007, 'id_str': '2464393007', 'name': 'Jordan Hiraldo', 'screen_name': 'JordanHiraldo', 'location': 'New York, NY', 'url': 'http://instagram.com/jordanhiraldo', 'description': 'WEEKEND WARRIORS CEO // More content on my Instagram', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 574, 'friends_count': 1220, 'listed_count': 2, 'favourites_count': 4725, 'statuses_count': 4388, 'created_at': 'Sun Apr 06 08

{'created_at': 'Thu Apr 23 21:17:53 +0000 2020', 'id': 1253432934465335296, 'id_str': '1253432934465335296', 'text': 'Boy is that ketchup directly on the fries', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 89598185, 'id_str': '89598185', 'name': 'Taco Connoisseur', 'screen_name': 'MyKeyTailor', 'location': 'Dallas, TX', 'url': 'http://taylordtimes.com', 'description': 'Engineer, Traveler, Chef, Photographer, GOMAB. Faizon Love parody account.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1614, 'friends_count': 878, 'listed_count': 20, 'favourites_count': 20640, 'statuses_count': 221075, 'created_at': 'Fri Nov 13 01:49:27 +0000 2009', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': N

{'created_at': 'Thu Apr 23 21:17:54 +0000 2020', 'id': 1253432935132229632, 'id_str': '1253432935132229632', 'text': 'https://t.co/jjzZfNrWa9 https://t.co/7aOPfdqzOO', 'display_text_range': [0, 23], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 783696509711683584, 'id_str': '783696509711683584', 'name': 'Taft CCM Ctr', 'screen_name': 'Taft_Career_Ctr', 'location': 'San Antonio, TX', 'url': 'https://sites.google.com/a/nisd.net/college-career-center/', 'description': 'Mrs. Torres\nTaft HS Alumni', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 305, 'friends_count': 381, 'listed_count': 0, 'favourites_count': 4127, 'statuses_count': 1565, 'created_at': 'Wed Oct 05 15:53:03 +0000 2016', 'utc_offset': None, 't

{'created_at': 'Thu Apr 23 21:17:54 +0000 2020', 'id': 1253432936390488066, 'id_str': '1253432936390488066', 'text': '@ohdatstay Why did my mom ask me the other day when do I plan on getting pregnant 💀💀💀', 'display_text_range': [11, 85], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1253430941210443778, 'in_reply_to_status_id_str': '1253430941210443778', 'in_reply_to_user_id': 1047992504, 'in_reply_to_user_id_str': '1047992504', 'in_reply_to_screen_name': 'ohdatstay', 'user': {'id': 3403146598, 'id_str': '3403146598', 'name': 'Nyla✨', 'screen_name': 'NylaChann', 'location': None, 'url': None, 'description': '20| ODU| MUA💓', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 442, 'friends_count': 386, 'listed_count': 0, 'favourites_count': 726, 'statuses_count': 1262, 'created_at': 'Tue Aug 04 18:38:21 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_e

...

{'created_at': 'Thu Apr 23 21:18:51 +0000 2020', 'id': 1253433177059672064, 'id_str': '1253433177059672064', 'text': '@GovMikeDeWine @OSUPublicHealth 🤬', 'display_text_range': [32, 33], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1253398210703474688, 'in_reply_to_status_id_str': '1253398210703474688', 'in_reply_to_user_id': 18820392, 'in_reply_to_user_id_str': '18820392', 'in_reply_to_screen_name': 'GovMikeDeWine', 'user': {'id': 261201848, 'id_str': '261201848', 'name': 'Ronperk', 'screen_name': 'ronperk643', 'location': 'Willoughby Ohio', 'url': None, 'description': "If you don't wear a Brown and Orange N95 mask, you don't matter. Ugh...nevermind.", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 134, 'friends_count': 533, 'listed_count': 2, 'favourites_count': 7777, 'statuses_count': 6554, 'created_at': 'Sat Mar 05 13:19:52 +0000 2011', 'utc_offset'

{'created_at': 'Thu Apr 23 21:18:51 +0000 2020', 'id': 1253433177751777286, 'id_str': '1253433177751777286', 'text': 'Same! @SteveMartinToGo in Kansas City. Sadly, The Blues Brothers were not the opening act. I don’t even know if I c… https://t.co/RkplQUOpnb', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 79242049, 'id_str': '79242049', 'name': 'Jon Michael Davis', 'screen_name': 'JonMichaelDavis', 'location': 'United States', 'url': 'http://jonmichaeldavis.com', 'description': 'Sometimes an actor. Sometimes a voiceover guy. Sometimes a martial arts instructor. Always early • Founder @ZenHapkido • http://Facebook.com/RUNJMD GenX AF Ⓥ', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1542, 'friends_count': 308

{'created_at': 'Thu Apr 23 21:18:52 +0000 2020', 'id': 1253433178959691780, 'id_str': '1253433178959691780', 'text': '‘In My Head’ available May 1st!! ♥️ #newmusic #comingsoon #denverartist #inmyhead @ Denver, Colorado https://t.co/3vUrIFdn9f', 'source': '<a href="http://instagram.com" rel="nofollow">Instagram</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 22605094, 'id_str': '22605094', 'name': 'Zanya Laurence', 'screen_name': 'zanyalaurence', 'location': 'Colorado', 'url': 'http://www.zanyamusic.com', 'description': 'Singer/ Songwriter/ Artist/ Human. Follow on IG: @zanyalaurence YT: zanyalaurence FB: zanyalaurence Visit website for more info ♥', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 647, 'friends_count': 650, 'listed_count': 17, 'favourites_count': 1268, 'statuses_count': 1061, 'creat

{'created_at': 'Thu Apr 23 21:18:52 +0000 2020', 'id': 1253433179966279688, 'id_str': '1253433179966279688', 'text': 'No se si decirle gracias o te odio a esta cuarentena', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1624326085, 'id_str': '1624326085', 'name': 'Agostina 🃏', 'screen_name': 'agostinadlc_f', 'location': 'North Miami Beach, FL', 'url': None, 'description': 'La belleza está en tu cabeza', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 976, 'friends_count': 741, 'listed_count': 1, 'favourites_count': 2322, 'statuses_count': 13721, 'created_at': 'Sat Jul 27 01:43:27 +0000 2013', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translato

{'created_at': 'Thu Apr 23 21:18:52 +0000 2020', 'id': 1253433180817747970, 'id_str': '1253433180817747970', 'text': '@GovBillLee @MariaLeeTN Yeah well we just hit the 8000 case mark out of 8,000,000 and you are basically telling us… https://t.co/ABWatrB20o', 'display_text_range': [24, 140], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': 1253404876891095047, 'in_reply_to_status_id_str': '1253404876891095047', 'in_reply_to_user_id': 1084924525232513025, 'in_reply_to_user_id_str': '1084924525232513025', 'in_reply_to_screen_name': 'GovBillLee', 'user': {'id': 2741453566, 'id_str': '2741453566', 'name': 'Christian Hillbilly', 'screen_name': 'JlJlane45', 'location': 'east tn', 'url': None, 'description': 'Republican Hillbilly Artist devoted to our lord Jesus Christ and my familyand my country.Spread love not hate💒No lists No Trolls!', 'translator_type': 'none', 'protected': False, 'verified': Fals